In [1]:
# muzero in a notebook on cartpole

In [62]:
%pylab inline
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import gym
from tqdm import trange
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import *
np.set_printoptions(suppress=True)

def toh(x,n):
    ret = np.zeros([n])
    ret[x] = 1.0
    return ret

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'concatenate', 'subtract', 'multiply', 'dot', 'maximum', 'minimum', 'gamma', 'add', 'average']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
env = gym.make("CartPole-v0")
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [81]:
S_DIM = 4

# h: representation function
# s_0 = h(o_1...o_t)
x = o_0 = Input(env.observation_space.shape)
x = Dense(S_DIM)(x)
s_0 = Activation('tanh')(x)
h = Model(o_0, s_0)
def ht(o_0):
    return h.predict(o_0[None])[0]

# g: dynamics function (recurrent in state?) old_state+action -> state+reward
# r_k, s_k = g(s_k-1, a_k)
s_km1 = Input(S_DIM)
a_k = Input(env.action_space.n)
x = Concatenate()([s_km1, a_k])
x = Dense(64)(x)
x = Activation('elu')(x)
x = Dense(64)(x)
x = Activation('elu')(x)
s_k = Dense(S_DIM, name='s_k')(x)
r_k = Dense(1, name='r_k')(x)
g = Model([s_km1, a_k], [r_k, s_k])
g.compile('adam', 'mse')
def gt(s_km1, a_k):
    r_k, s_k = g.predict([s_km1[None], a_k[None]])
    return r_k[0], s_k[0]

# f: prediction function -- state -> policy+value
# p_k, v_k = f(s_k)
x = s_k = Input(S_DIM)
x = Dense(32)(x)
x = Activation('tanh')(x)
p_k = Dense(env.action_space.n)(x)
p_k = Activation('softmax', name='p_k')(p_k)
v_k = Dense(1, name='v_k')(x)
f = Model(s_k, [p_k, v_k])
def ft(s_k):
    p_k, v_k = f.predict(s_k[None])
    return p_k[0], v_k[0]

In [118]:
# it's using the dynamics function for rollout search
K = 5
gamma = 0.997

# represent
o_0 = Input(env.observation_space.shape)
s_km1 = h(o_0)

# rollout with dynamics
# p_k, v_k, r_k = mu(o_0, a_1_k)
a_all, mu_all = [], []
for k in range(K):
    a_k = Input(env.action_space.n)
    r_k, s_k  = g([s_km1, a_k])
    
    # predict
    p_k, v_k = f([s_k])
    
    # store
    a_all.append(a_k)
    mu_all.append([p_k, v_k, r_k])
    s_km1 = s_k

# put in the first observation and actions
#   need policy from search
#   need values from sum of rewards + last state value (real state?)
#   need rewards
#a_all = Concatenate()(a_all)
mu = Model([o_0, a_all], mu_all)
mu.compile('adam', 'mse')

In [156]:
import itertools

aopts = list(itertools.product([0,1], repeat=K))
aoptss = np.array([[toh(x, 2) for x in aa] for aa in aopts])
aoptss = aoptss.swapaxes(0,1)
aoptss = [aoptss[x] for x in range(5)]

def search(o_0):
    # 2^5 rollout isn't too bad, explore them all
    # maximize value
    o_0s = np.repeat(np.array(o_0)[None], len(aopts), axis=0)
    ret = mu.predict([o_0s]+aoptss)
    v = [(ret[i][-2][0], aopts[i]) for i in range(len(ret))]
    return sorted(v, reverse=True)
    
search(env.state)

[(0.5505225, (0, 1, 1, 0, 0)),
 (0.5449144, (0, 0, 0, 0, 0)),
 (0.5441888, (0, 1, 0, 0, 1)),
 (0.5422417, (0, 0, 1, 1, 0)),
 (0.5402329, (0, 0, 0, 1, 1)),
 (0.26654688, (0, 1, 1, 0, 1)),
 (0.1320413, (0, 1, 0, 1, 0)),
 (0.12378375, (0, 0, 0, 1, 0)),
 (0.11833585, (0, 0, 1, 1, 1)),
 (0.088117436, (0, 0, 1, 0, 0)),
 (0.083467014, (0, 0, 0, 0, 1)),
 (0.05097529, (0, 1, 1, 1, 0)),
 (-0.07591396, (0, 0, 1, 0, 1)),
 (-0.08704735, (0, 1, 0, 0, 0)),
 (-0.10349347, (0, 1, 0, 1, 1))]

In [155]:
x,y = [],[]
for _ in trange(8):
    env.reset()
    o_0 = np.copy(env.state)
    acts = search(o_0)
    v_k, a_0k = acts[0]
    a_0koh = [toh(x, 2) for x in a_0k]
    x.append([o_0]+a_0koh)
    
    # actually act with best value policy
    rs = []
    for i in range(K):
        _, r, done, _ = env.step(a_0k[i])
        rs.append(r)
    y.append([a_0koh, v_k, rs])
x,y

100%|██████████| 8/8 [00:00<00:00, 37.91it/s]


([[array([-0.04993264,  0.04836301, -0.04479264,  0.02209651]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([ 0.04475481, -0.03243983,  0.00393763, -0.04384957]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([-0.03185653,  0.04431162, -0.01077688, -0.02931607]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([-0.01931571,  0.04927027,  0.02496572,  0.04038208]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([-0.0198104 ,  0.02291996,  0.02285041, -0.00901822]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([ 0.01769356, -0.02794923,  0.02159403,  0.02309088]),
   array([1., 0.]),
   array([0., 1.]),
   array([0., 1.]),
   array([1., 0.]),
   array([1., 0.])],
  [array([

In [89]:
mu.inputs

[<tf.Tensor 'input_57:0' shape=(None, 4) dtype=float32>,
 <tf.Tensor 'input_58:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'input_59:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'input_60:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'input_61:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'input_62:0' shape=(None, 2) dtype=float32>]

In [88]:
mu.outputs

[<tf.Tensor 'model_20/Identity:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'model_20/Identity_1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_19/Identity:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_20_1/Identity:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'model_20_1/Identity_1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_19_1/Identity:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_20_2/Identity:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'model_20_2/Identity_1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_19_2/Identity:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_20_3/Identity:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'model_20_3/Identity_1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_19_3/Identity:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_20_4/Identity:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'model_20_4/Identity_1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'model_19_4/Identity:0' shape=(N

In [63]:
# Test: Can the g function learn a dynamics model
# Answer: yes!

x_s, x_a = [], []
y_r, y_s = [], []

for _ in trange(1000):
    env.reset()
    for _ in range(100):
        #env.render()
        a_k = env.action_space.sample()
        x_s.append(env.state)
        x_a.append(toh(a_k, env.action_space.n))
        _, r, done, _ = env.step(a_k)
        y_r.append([r])
        y_s.append(env.state)
        #if done:
        #    break
    env.close()
x_s, x_a = np.array(x_s), np.array(x_a)
y_r, y_s = np.array(y_r), np.array(y_s)

g.fit([x_s, x_a], [y_r, y_s], batch_size=32, epochs=5, shuffle=1)

env.reset()
for _ in range(100):
    env.render()
    a_k = env.action_space.sample()
    x1 = np.array(env.state)[None]
    x2 = toh(a_k, env.action_space.n)[None]
    r_k, s_k = g.predict([x1, x2])
    print(r_k)
    env.unwrapped.state = s_k[0]
    #env.step(a_k)
env.close()

100%|██████████| 1000/1000 [00:02<00:00, 418.91it/s]


Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 3s 34us/sample - loss: 0.0156 - r_k_loss: 0.0115 - s_k_loss: 0.0040
Epoch 2/5
100000/100000 [==============================] - 3s 34us/sample - loss: 0.0146 - r_k_loss: 0.0111 - s_k_loss: 0.0036
Epoch 3/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0143 - r_k_loss: 0.0110 - s_k_loss: 0.0033
Epoch 4/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0138 - r_k_loss: 0.0107 - s_k_loss: 0.0031
Epoch 5/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0136 - r_k_loss: 0.0106 - s_k_loss: 0.0030
[[1.081461]]
[[1.0875441]]
[[1.0959835]]
[[1.0935487]]
[[1.0834298]]
[[1.0871282]]
[[1.0947123]]
[[1.1018445]]
[[1.0914918]]
[[1.0920722]]
[[1.093135]]
[[1.0658582]]
[[1.0645722]]
[[1.0340444]]
[[1.0189047]]
[[1.0192426]]
[[1.0456619]]
[[1.0602669]]
[[1.0588717]]
[[0.9857284]]
[[0.71624035]]
[[0.37024945]]
[[0.23297256]]
[[0.2115975]]
[[0.